In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/code/siumaai/examples/msra_ner')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!nvidia-smi


Fri Jul  8 14:02:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# config environment

In [2]:
import os
os.chdir('/content/drive/MyDrive/code/siumaai/examples/msra_ner')

In [3]:
import os
import sys
root_path = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd())))), 'siumaai')
sys.path.append(root_path)



In [4]:
!pip install transformers pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 28.2 MB/s 
     |████████████████████████████████| 585 kB 19.3 MB/s 
     |████████████████████████████████| 596 kB 71.2 MB/s 
     |████████████████████████████████| 6.6 MB 74.5 MB/s 
     |████████████████████████████████| 101 kB 12.8 MB/s 
     |████████████████████████████████| 419 kB 69.1 MB/s 
     |████████████████████████████████| 140 kB 67.2 MB/s 
     |████████████████████████████████| 1.1 MB 64.7 MB/s 
     |████████████████████████████████| 271 kB 78.7 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 144 kB 72.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
import json
import torch
from dataclasses import asdict
from torch.utils.data import random_split, DataLoader
from siumaai.features.ner.bio import convert_logits_to_examples, convert_crf_logits_to_examples
from torch.utils.data.dataloader import default_collate
from siumaai.features.ner.bio import BIOForNerDataset
from siumaai.features.ner import EntityExample, NerExample
from transformers import BertTokenizerFast
import pytorch_lightning as pl
from siumaai.pl_models.ner import CrfNer
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from transformers import AutoConfig
from siumaai.models import MODEL_CLS_MAP


In [6]:
pl.seed_everything(2)


Global seed set to 2


2

# load label

In [7]:
MAX_SEQ_LENGTH=128
# PRETRAIN_MODEL_PATH='/home/zz/data/models/albert/albert_chinese_tiny'
PRETRAIN_MODEL_PATH='bert-base-chinese'

In [8]:
ID_TO_LABEL_MAP = {}
LABEL_TO_ID_MAP = {}
with open('msra/ner/labels.txt', encoding='utf-8')as f:
    index = 0
    for line in f:
        label = line.strip()
        if label and label != 'O':
            ID_TO_LABEL_MAP[index] = f'B-{label}'
            LABEL_TO_ID_MAP[f'B-{label}'] = index
            index += 1

            ID_TO_LABEL_MAP[index] = f'I-{label}'
            LABEL_TO_ID_MAP[f'I-{label}'] = index
            index += 1

        elif label and label == 'O':
            ID_TO_LABEL_MAP[index] = label
            LABEL_TO_ID_MAP[label] = index
            index += 1

ID_TO_LABEL_MAP[len(ID_TO_LABEL_MAP)] = '[PAD]'
LABEL_TO_ID_MAP['[PAD]'] = len(LABEL_TO_ID_MAP)

NUM_LABELS = len(ID_TO_LABEL_MAP)
PAD_ID = LABEL_TO_ID_MAP['[PAD]']
# PAD_ID = -100

print(f'id_to_label_map: {ID_TO_LABEL_MAP}')
print(f'label_to_id_map: {LABEL_TO_ID_MAP}')


id_to_label_map: {0: 'B-ns', 1: 'I-ns', 2: 'B-nt', 3: 'I-nt', 4: 'O', 5: 'B-nr', 6: 'I-nr', 7: '[PAD]'}
label_to_id_map: {'B-ns': 0, 'I-ns': 1, 'B-nt': 2, 'I-nt': 3, 'O': 4, 'B-nr': 5, 'I-nr': 6, '[PAD]': 7}


# load data

In [9]:
with open('msra/ner/data.json', encoding='utf-8') as f:
    example_list = [
        NerExample(
            text=data['text'],
            words=list(data['text']),
            entities=[
                EntityExample(
                    start_idx=entity['start_idx'],
                    end_idx=entity['end_idx'],
                    entity=entity['entity'],
                    type=entity['label']
                )
                for entity in data.get('entities', [])
            ]
        )
        for data in json.load(f)
    ]

train_example_size = int(len(example_list) * 0.8)
val_example_size = int(len(example_list) * 0.1)
test_example_size = len(example_list) - train_example_size - val_example_size
train_example_list, val_example_list, test_example_list = random_split(
        example_list, [train_example_size, val_example_size, test_example_size])


# load tokenizer

In [10]:
tokenizer = BertTokenizerFast.from_pretrained(PRETRAIN_MODEL_PATH)
tokenizer.add_special_tokens({'additional_special_tokens': [' ', '\n']})


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

2

# train

## 1. load train_data, val_data

In [11]:
BATCH_SIZE = 120
train_dataset = BIOForNerDataset(train_example_list, tokenizer, LABEL_TO_ID_MAP, MAX_SEQ_LENGTH, pad_id=PAD_ID, check_tokenization=False)
val_dataset = BIOForNerDataset(val_example_list, tokenizer, LABEL_TO_ID_MAP, MAX_SEQ_LENGTH, pad_id=PAD_ID, check_tokenization=False)

print(f'train_dataset_size: {len(train_dataset)}')
print(f'val_dataset_size: {len(val_dataset)}')


def fit_collate_func(batch):
    return default_collate([
        {
            'input_ids': data.input_ids,
            'attention_mask': data.attention_mask,
            'token_type_ids': data.token_type_ids,
            'labels': data.labels
        }
        for data in batch
    ])

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=fit_collate_func)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=fit_collate_func)


train_dataset_size: 40583
val_dataset_size: 5072


## 2. init model

In [12]:
config = AutoConfig.from_pretrained(
    PRETRAIN_MODEL_PATH, 
    return_dict=None)

model_cls = MODEL_CLS_MAP['crf_for_ner']
model_kwargs = {
    'pretrain_model_path': PRETRAIN_MODEL_PATH,
    'num_labels': NUM_LABELS,
    'dropout_rate': config.hidden_dropout_prob,
    'hidden_size': config.hidden_size,
    'vocab_len': len(tokenizer)
}
# model = Ner(
model = CrfNer(
        # crf_learning_rate=0.005248074602497723,
        # learning_rate=0.0005248074602497723,
        crf_learning_rate=1.9054607179632464e-04,
        learning_rate=1.9054607179632464e-05,
        adam_epsilon=1e-8,
        warmup_rate=0.1,
        weight_decay=0.1,
        model_cls=model_cls,
        **model_kwargs
        )


Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## 3. init trainer

In [13]:
trainer = Trainer(
        gpus=1,
        max_epochs=20,
        weights_summary=None,
        logger=TensorBoardLogger('tensorboard_logs/crf'),
        callbacks=[
            EarlyStopping(
                monitor='val_loss',
                min_delta=0.1,
                patience=5,
                verbose=False,
                mode='min'),
            ModelCheckpoint(
                dirpath='ckpt/crf',
                filename='{epoch}-{val_loss:.2f}',
                monitor='val_loss',
                mode='min',
                verbose=True,
                save_top_k=1),
            LearningRateMonitor(logging_interval='step')])





/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:172: LightningDeprecationWarning: Setting `Trainer(weights_summary=None)` is deprecated in v1.5 and will be removed in v1.7. Please set `Trainer(enable_model_summary=False)` instead.
  "Setting `Trainer(weights_summary=None)` is deprecated in v1.5 and will be removed"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


## 4. tune initial learning_rate

In [14]:
# lr = trainer.tuner.lr_find(model, train_dataloader, val_dataloader, early_stop_threshold=None)
# print(lr.suggestion())
# model.hparams.learning_rate = lr.suggestion()

## 5. training

In [ ]:
trainer.fit(model, train_dataloader, val_dataloader)

Missing logger folder: tensorboard_logs/crf/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
Loading `train_dataloader` to estimate number of stepping batches.


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:498: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  category=PossibleUserWarning,
/content/drive/MyDrive/code/siumaai/siumaai/layers/crf.py:233: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  ../aten/src/ATen/native/TensorCompare.cpp:333.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Epoch 1, global step 678: 'val_loss' reached 1.21126 (best 1.21126), saving model to '/content/drive/MyDrive/code/siumaai/examples/msra_ner/ckpt/crf/epoch=1-val_loss=1.21.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

Epoch 2, global step 1017: 'val_loss' reached 0.98797 (best 0.98797), saving model to '/content/drive/MyDrive/code/siumaai/examples/msra_ner/ckpt/crf/epoch=2-val_loss=0.99.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

Epoch 3, global step 1356: 'val_loss' reached 0.94693 (best 0.94693), saving model to '/content/drive/MyDrive/code/siumaai/examples/msra_ner/ckpt/crf/epoch=3-val_loss=0.95.ckpt' as top 1


# test

## 1. load test data

In [ ]:
test_dataset = BIOForNerDataset(test_example_list, tokenizer, LABEL_TO_ID_MAP, MAX_SEQ_LENGTH, pad_id=PAD_ID, check_tokenization=False)


## 2. load model

In [ ]:
model = CrfNer.load_from_checkpoint('ckpt/epoch=3-val_loss=0.94.ckpt')
model.to('cuda')
model.eval()


## 3. testing

In [ ]:
TEST_BATCH_SIZE = 1000
pred_example_list = []
crf_pred_example_list = []
start_index = 0
while start_index < len(test_dataset):
    if start_index + TEST_BATCH_SIZE < len(test_dataset):
        end_index  = start_index + TEST_BATCH_SIZE 
    else:
        end_index = len(test_dataset)

    feature_list = []
    batch = []
    for index in range(start_index, end_index):
        feature_list.append(test_dataset[index])
        batch.append({
            'input_ids': test_dataset[index].input_ids,
            'attention_mask': test_dataset[index].attention_mask,
            'token_type_ids': test_dataset[index].token_type_ids,
        })

    #crf_logits, logits, *_ = model(**default_collate(batch))
    batch = {k:v.to('cuda') for k,v in default_collate(batch).items()}
    with torch.no_grad():
        crf_logits, logits = model(**batch)
        crf_logits = crf_logits.detach().cpu()
        logits = logits.detach().cpu()
    pred_example_list.extend(convert_logits_to_examples(feature_list, logits, ID_TO_LABEL_MAP))
    crf_pred_example_list.extend(convert_crf_logits_to_examples(feature_list, crf_logits, ID_TO_LABEL_MAP))
    print(f'finish {start_index} -> {end_index}')
    start_index = end_index


In [ ]:
import torch
torch.cuda.empty_cache()

# metric

In [ ]:
from siumaai.metrics.ner import calc_metric
metric = calc_metric(test_example_list, pred_example_list)
print(metric)

crf_metric = calc_metric(test_example_list, crf_pred_example_list)
print(crf_metric)
